# Average Interactions a user had on your website before making a purchase

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# BigQuery SDK Library setup
%load_ext google.cloud.bigquery

import os
from google.cloud import bigquery
from google.oauth2 import service_account


The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


## Load Google Analytics 360 data from BigQuery

In [ ]:
def load_data_from_bq(sql, credential_json_file):

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=credential_json_file
    credentials = service_account.Credentials.from_service_account_file(credential_json_file)

    bqclient = bigquery.Client(credentials=credentials)

    # Download query results.
    query_string = sql

    df = (
        bqclient.query(query_string)
            .result()
            .to_dataframe(
            # Optionally, explicitly request to use the BigQuery Storage API. As of
            # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
            # API is used by default.
            create_bqstorage_client=True,
        )
    )

    return df

In [ ]:
my_ga_dataset = "bigquery-public-data.google_analytics_sample"  # Replace with your own GA 360 BigQuery dataset

credentials = "credentials.json". # Replace this with your own Bigquery Service Credentials JSON

sql = f"""
SELECT
  Table_1.productSku AS ProductSku,
  Table_1.v2ProductName AS ProductName,
  ( sum_of_hit_number / total_hits ) AS avg_hits
FROM (
  SELECT
    p.productSku,
    p.v2ProductName,
    SUM(h.hitNumber) AS sum_of_hit_number
  FROM
    `{my_ga_dataset}.ga_sessions_*`,
    UNNEST(hits) AS h,
    UNNEST(product) AS p
  WHERE
    totals.transactions>=1
    AND p.productSku IN (
    SELECT
      DISTINCT p.productSku
    FROM
      `{my_ga_dataset}.ga_sessions_*`,
      UNNEST(hits) AS h,
      UNNEST(product) AS p
    WHERE
      p.productSku IS NOT NULL)
  GROUP BY
    productSku,
    v2ProductName ) AS Table_1
JOIN (
  SELECT
    p.productSku AS productSku,
    COUNT( fullVisitorId ) AS total_hits
  FROM
    `{my_ga_dataset}.ga_sessions_*`,
    UNNEST(hits) AS h,
    UNNEST(product) AS p
  WHERE
    p.productSku IS NOT NULL
    AND totals.transactions>=1
  GROUP BY
    p.productSku ) AS Table_2
ON
  Table_1.productSku = Table_2.productSku;
"""

df = load_data_from_bq(sql=sql, credential_json_file=credentials)

In [ ]:
df.head()

,ProductSku,ProductName,avg_hits
0,GGOEGAAX0657,Google Toddler 1/4 Zip Fleece Pewter,26.484218
1,GGOEYOBR078599,YouTube Luggage Tag,24.547529
2,GGOEYOLR080599,YouTube Notebook and Pen Set,22.692982
3,9184620,Google Men's Performance Full Zip Jacket Black,31.418605
4,GGOEA0CH077599,Android Hard Cover Journal,24.978102


## Slow-moving Products: Customers spend lot of time on them before buying

In [ ]:
df.sort_values('avg_hits', ascending=False).head(10)

,ProductSku,ProductName,avg_hits
1977,GGOEAAEB028315,Android Women's Short Sleeve Hero Tee Black,303.500000
1916,GGOEYAEB031416,YouTube Men's 3/4 Sleeve Henley,249.600000
2036,GGOEGALQ058113,Google Women's Colorblock Tee White,249.250000
417,GGOEGAAJ080713,Google Men's Around The Block Short Sleeve Tee...,248.625000
1339,GGOEGAEJ030717,Google Men's 3/4 Sleeve Raglan Henley Grey,241.000000
2015,GGOEAAEB031214,Android Heavyweight Long Sleeve Badge Tee Black,238.573171
828,9180774,Pet Feeding Mat,200.000000
1617,GGOEGAXC075155,Google Toddler Tee Fruit Games Lemon,191.500000
1299,GGOEAAEB031616,Android Men's 3/4 Sleeve Raglan Henley Black,184.304348
360,GGOEAAAJ034513,Android Men's Vintage Tee,182.560000


## Fast-moving Products: Customers promptly buy them without doing much research on them

In [ ]:
df.sort_values('avg_hits', ascending=True).head(10)

,ProductSku,ProductName,avg_hits
371,GGOEGAAX0127,Android 24 oz Button Lid Sport Water Bottle,0.027006
1319,GGOEGDWR015799,15 oz Ceramic Mug,0.031949
802,GGOEGBMC056599,Google Small Waterproof Duffel,0.057475
1783,GGOEGESQ016799,Google Magnetic Mini Flood Flashlight,0.060445
1997,GGOEGAAX0327,Google Men's Long & Lean Tee Grey,0.070234
1992,GGOEGOXQ016399,Google Badge Pull,0.077317
510,GGOEGAAX0098,Google 7-inch Dog Flying Disc,0.079084
722,9182575,Android Men's Zip Hoodie,0.079710
1018,GGOEGAAX0328,Google Men's Long & Lean Tee Charcoal,0.082099
1304,GGOEGHGH019699,Google Sunglasses Green,0.083260


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0bf64fc0-3489-421f-847e-b901bd3270a1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>